<a href="https://colab.research.google.com/github/jacquesescp/DSforAM_group10/blob/main/ReportGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
"""
Risk Management PDF Report Generator Module

This module generates comprehensive PDF reports summarizing portfolio risks
for investors and compliance entities.

Requirements:
    pip install reportlab pandas matplotlib
"""

from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import (
    SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer,
    PageBreak, Image, KeepTogether
)
from reportlab.lib.enums import TA_CENTER, TA_RIGHT, TA_JUSTIFY
from reportlab.pdfgen import canvas
from datetime import datetime
from typing import Dict, List, Optional, Union
import matplotlib.pyplot as plt
import io
import os


class RiskReportGenerator:
    """
    Generates PDF reports for portfolio risk analysis.

    Attributes:
        portfolio_name (str): Name of the portfolio
        report_date (datetime): Date of the report
        output_path (str): Path where PDF will be saved
    """

    def __init__(self, portfolio_name: str, output_path: str = "risk_report.pdf"):
        """
        Initialize the report generator.

        Args:
            portfolio_name: Name of the portfolio being analyzed
            output_path: File path for the output PDF
        """
        self.portfolio_name = portfolio_name
        self.report_date = datetime.now()
        self.output_path = output_path
        self.styles = getSampleStyleSheet()
        self._setup_custom_styles()

    def _setup_custom_styles(self):
        """Setup custom paragraph styles for the report."""
        # Title style
        self.styles.add(ParagraphStyle(
            name='CustomTitle',
            parent=self.styles['Heading1'],
            fontSize=24,
            textColor=colors.HexColor('#1a1a1a'),
            spaceAfter=30,
            alignment=TA_CENTER,
            fontName='Helvetica-Bold'
        ))

        # Section header style
        self.styles.add(ParagraphStyle(
            name='SectionHeader',
            parent=self.styles['Heading2'],
            fontSize=16,
            textColor=colors.HexColor('#2c3e50'),
            spaceAfter=12,
            spaceBefore=12,
            fontName='Helvetica-Bold'
        ))

        # Risk level style
        self.styles.add(ParagraphStyle(
            name='RiskLevel',
            parent=self.styles['Normal'],
            fontSize=14,
            spaceAfter=6,
            fontName='Helvetica-Bold'
        ))

    def _create_header_footer(self, canvas, doc):
        """Create header and footer for each page."""
        canvas.saveState()

        # Header
        canvas.setFont('Helvetica-Bold', 10)
        canvas.drawString(inch, letter[1] - 0.5*inch,
                         f"Portfolio: {self.portfolio_name}")
        canvas.drawRightString(letter[0] - inch, letter[1] - 0.5*inch,
                              f"Report Date: {self.report_date.strftime('%Y-%m-%d')}")

        # Footer
        canvas.setFont('Helvetica', 8)
        canvas.drawString(inch, 0.5*inch, "CONFIDENTIAL - For Authorized Use Only")
        canvas.drawRightString(letter[0] - inch, 0.5*inch,
                              f"Page {doc.page}")

        canvas.restoreState()

    def _get_risk_color(self, risk_level: str) -> colors.Color:
        """
        Get color based on risk level.

        Args:
            risk_level: Risk level (Low, Medium, High, Critical)

        Returns:
            Color object for the risk level
        """
        risk_colors = {
            'Low': colors.HexColor('#27ae60'),
            'Medium': colors.HexColor('#f39c12'),
            'High': colors.HexColor('#e67e22'),
            'Critical': colors.HexColor('#c0392b')
        }
        return risk_colors.get(risk_level, colors.grey)

    def _create_summary_table(self, risk_data: Dict[str, Dict]) -> Table:
        """
        Create summary table of all risk types.

        Args:
            risk_data: Dictionary containing risk metrics for each category

        Returns:
            Table object with risk summary
        """
        data = [['Risk Category', 'Level', 'Score', 'Status']]

        for category, metrics in risk_data.items():
            level = metrics.get('level', 'N/A')
            score = metrics.get('score', 'N/A')
            status = metrics.get('status', 'N/A')

            data.append([
                category,
                level,
                f"{score:.2f}" if isinstance(score, (int, float)) else score,
                status
            ])

        table = Table(data, colWidths=[2.5*inch, 1.5*inch, 1.5*inch, 2*inch])
        table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#34495e')),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 12),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
            ('GRID', (0, 0), (-1, -1), 1, colors.black),
            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 1), (-1, -1), 10),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.lightgrey])
        ]))

        return table

    def _create_detailed_risk_table(self, risk_category: str,
                                   metrics: Dict[str, Union[float, str]]) -> Table:
        """
        Create detailed table for a specific risk category.

        Args:
            risk_category: Name of the risk category
            metrics: Dictionary of metrics for this category

        Returns:
            Table object with detailed metrics
        """
        data = [['Metric', 'Value']]

        for metric, value in metrics.items():
            if metric not in ['level', 'status']:
                formatted_value = f"{value:.4f}" if isinstance(value, float) else str(value)
                data.append([metric.replace('_', ' ').title(), formatted_value])

        table = Table(data, colWidths=[3*inch, 3*inch])
        table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2c3e50')),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 11),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 10),
            ('GRID', (0, 0), (-1, -1), 1, colors.black),
            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 1), (-1, -1), 9),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.lightgrey])
        ]))

        return table

    def _create_risk_chart(self, risk_data: Dict[str, Dict]) -> Optional[Image]:
        """
        Create a bar chart visualization of risk scores.

        Args:
            risk_data: Dictionary containing risk metrics

        Returns:
            Image object containing the chart
        """
        categories = []
        scores = []
        colors_list = []

        for category, metrics in risk_data.items():
            if 'score' in metrics and isinstance(metrics['score'], (int, float)):
                categories.append(category)
                scores.append(metrics['score'])
                level = metrics.get('level', 'Medium')
                # Convert reportlab color to hex for matplotlib, ensuring '#' prefix
                reportlab_color = self._get_risk_color(level)
                hex_color = reportlab_color.hexval()
                # Ensure the hex string starts with '#' for matplotlib
                if hex_color.startswith('0x'):
                    hex_color = '#' + hex_color[2:]
                elif not hex_color.startswith('#'):
                     hex_color = '#' + hex_color
                colors_list.append(hex_color)


        if not scores:
            return None

        fig, ax = plt.subplots(figsize=(8, 4))
        bars = ax.barh(categories, scores, color=colors_list)
        ax.set_xlabel('Risk Score', fontsize=10, fontweight='bold')
        ax.set_title('Risk Assessment Overview', fontsize=12, fontweight='bold')
        ax.set_xlim(0, max(scores) * 1.2 if scores else 10)

        # Add value labels on bars
        for bar, score in zip(bars, scores):
            width = bar.get_width()
            ax.text(width, bar.get_y() + bar.get_height()/2,
                   f'{score:.2f}', ha='left', va='center', fontsize=9)

        plt.tight_layout()

        # Save to bytes
        img_buffer = io.BytesIO()
        plt.savefig(img_buffer, format='png', dpi=150, bbox_inches='tight')
        img_buffer.seek(0)
        plt.close()

        return Image(img_buffer, width=6*inch, height=3*inch)

    def generate_report(self,
                       credit_risk: Dict,
                       market_risk: Dict,
                       liquidity_risk: Dict,
                       operational_risk: Dict,
                       integrated_risk: Dict,
                       recommendations: Optional[List[str]] = None,
                       compliance_notes: Optional[str] = None) -> str:
        """
        Generate complete PDF risk report.

        Args:
            credit_risk: Credit risk metrics and assessment
            market_risk: Market risk metrics and assessment
            liquidity_risk: Liquidity risk metrics and assessment
            operational_risk: Operational risk metrics and assessment
            integrated_risk: Integrated risk metrics and assessment
            recommendations: List of risk mitigation recommendations
            compliance_notes: Additional compliance-related notes

        Returns:
            Path to the generated PDF file
        """
        doc = SimpleDocTemplate(
            self.output_path,
            pagesize=letter,
            rightMargin=inch,
            leftMargin=inch,
            topMargin=inch,
            bottomMargin=inch
        )

        story = []

        # Title Page
        story.append(Spacer(1, 2*inch))
        title = Paragraph(
            f"Risk Management Report<br/>{self.portfolio_name}",
            self.styles['CustomTitle']
        )
        story.append(title)
        story.append(Spacer(1, 0.5*inch))

        report_info = Paragraph(
            f"<b>Report Date:</b> {self.report_date.strftime('%B %d, %Y')}<br/>"
            f"<b>Report Type:</b> Comprehensive Risk Assessment<br/>"
            f"<b>Prepared for:</b> Investors and Compliance Entities",
            self.styles['Normal']
        )
        story.append(report_info)
        story.append(PageBreak())

        # Executive Summary
        story.append(Paragraph("Executive Summary", self.styles['SectionHeader']))
        story.append(Spacer(1, 0.2*inch))

        risk_data = {
            'Credit Risk': credit_risk,
            'Market Risk': market_risk,
            'Liquidity Risk': liquidity_risk,
            'Operational Risk': operational_risk,
            'Integrated Risk': integrated_risk
        }

        summary_table = self._create_summary_table(risk_data)
        story.append(summary_table)
        story.append(Spacer(1, 0.3*inch))

        # Risk visualization
        chart = self._create_risk_chart(risk_data)
        if chart:
            story.append(chart)

        story.append(PageBreak())

        # Detailed Risk Analysis
        story.append(Paragraph("Detailed Risk Analysis", self.styles['SectionHeader']))
        story.append(Spacer(1, 0.2*inch))

        for risk_name, risk_metrics in risk_data.items():
            # Risk category header
            level = risk_metrics.get('level', 'N/A')
            color = self._get_risk_color(level)

            risk_header = Paragraph(
                f"<b>{risk_name}</b> - Level: <font color='{color.hexval()}'>{level}</font>",
                self.styles['RiskLevel']
            )
            story.append(risk_header)

            # Risk description
            description = risk_metrics.get('description', 'No description available.')
            story.append(Paragraph(description, self.styles['Normal']))
            story.append(Spacer(1, 0.1*inch))

            # Detailed metrics table
            detail_table = self._create_detailed_risk_table(risk_name, risk_metrics)
            story.append(detail_table)
            story.append(Spacer(1, 0.3*inch))

        story.append(PageBreak())

        # Recommendations
        if recommendations:
            story.append(Paragraph("Risk Mitigation Recommendations",
                                 self.styles['SectionHeader']))
            story.append(Spacer(1, 0.2*inch))

            for i, rec in enumerate(recommendations, 1):
                rec_para = Paragraph(f"<b>{i}.</b> {rec}", self.styles['Normal'])
                story.append(rec_para)
                story.append(Spacer(1, 0.1*inch))

            story.append(Spacer(1, 0.3*inch))

        # Compliance Notes
        if compliance_notes:
            story.append(Paragraph("Compliance Notes", self.styles['SectionHeader']))
            story.append(Spacer(1, 0.2*inch))
            compliance_para = Paragraph(compliance_notes, self.styles['Normal'])
            story.append(compliance_para)

        # Build PDF
        doc.build(story, onFirstPage=self._create_header_footer,
                 onLaterPages=self._create_header_footer)

        return self.output_path


# Example usage
if __name__ == "__main__":
    # Sample risk data
    credit_risk_data = {
        'level': 'Medium',
        'score': 6.5,
        'status': 'Monitoring Required',
        'description': 'Credit risk is at acceptable levels with some exposure to counterparty defaults.',
        'default_probability': 0.025,
        'exposure_at_default': 1500000,
        'loss_given_default': 0.45,
        'expected_loss': 16875
    }

    market_risk_data = {
        'level': 'High',
        'score': 7.8,
        'status': 'Action Required',
        'description': 'Market risk is elevated due to high volatility in equity positions.',
        'value_at_risk_95': 125000,
        'value_at_risk_99': 185000,
        'beta': 1.35,
        'sharpe_ratio': 0.82
    }

    liquidity_risk_data = {
        'level': 'Low',
        'score': 3.2,
        'status': 'Satisfactory',
        'description': 'Portfolio maintains adequate liquidity buffers.',
        'current_ratio': 2.5,
        'quick_ratio': 1.8,
        'cash_coverage': 0.35
    }

    operational_risk_data = {
        'level': 'Medium',
        'score': 5.5,
        'status': 'Under Review',
        'description': 'Operational processes are adequate with minor control weaknesses.',
        'process_failures': 3,
        'system_downtime_hours': 2.5,
        'control_effectiveness': 0.88
    }

    integrated_risk_data = {
        'level': 'Medium',
        'score': 6.1,
        'status': 'Acceptable',
        'description': 'Overall risk profile is within acceptable parameters.',
        'composite_score': 6.1,
        'risk_adjusted_return': 0.075,
        'capital_adequacy_ratio': 0.15
    }

    recommendations = [
        "Reduce equity exposure by 10% to mitigate market risk.",
        "Implement additional credit screening procedures for new counterparties.",
        "Enhance operational controls in trade settlement processes.",
        "Maintain current liquidity buffers given market conditions.",
        "Consider hedging strategies for interest rate exposure."
    ]

    compliance_notes = """
    This portfolio complies with all regulatory capital requirements.
    Risk management framework aligns with Basel III standards.
    Stress testing indicates resilience under adverse scenarios.
    Regular monitoring and reporting procedures are in place.
    """

    # Generate report
    generator = RiskReportGenerator("Global Diversified Fund", "portfolio_risk_report.pdf")
    output_file = generator.generate_report(
        credit_risk=credit_risk_data,
        market_risk=market_risk_data,
        liquidity_risk=liquidity_risk_data,
        operational_risk=operational_risk_data,
        integrated_risk=integrated_risk_data,
        recommendations=recommendations,
        compliance_notes=compliance_notes
    )

    print(f"Risk report generated: {output_file}")

Risk report generated: portfolio_risk_report.pdf


In [2]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.0 MB/s eta 0:00:00
